Menu Logic
basically:
- option 1:
train model
display accuracy on test dataset
save model

- option 2:
load model (if not trained = error)


In [1]:
import numpy as np
import os
import random

import cv2

*notes*

some of the algorithm are brute forced, and will be revisited again for improvement.

In [2]:
### Load dataset

# will load the dataset 
def get_dataset(parent_path):
    train_image = []
    train_label = []

    test_image = 

    image_count = []
    # idx -> 
    for idx, label in enumerate(os.listdir(parent_path)):
        image_path = os.path.join(parent_path, label)
        count = 0
        for images in os.listdir(image_path):
            # read from path
            path2 = os.path.join(image_path, images)
            gray_img = cv2.imread(path2, 0) # read as grayscale
            
            image_datasets.append(gray_img)
            label_datasets.append(idx)
            count += 1
        image_count.append(count)
    # this will return the image path & its label in order from the datasets
    return image_datasets, label_datasets, image_count


## splitting into train, test
def split_datasets(images, labels, label_count):
    test_image = []
    test_label = []
    popped = 0 # berapa banyak image yang di pop (dimasukin test dataset)

    for idx, x in enumerate(label_count):
        # ini 25% dari banyak foto
        random_num = random.sample(range(0, x), quarter_label_size) # get 5 unique number from 0 -> num of images
        
        random_num.sort() # ini perlu di sort buat gampangin pop

        min = x*idx-popped # starting index of i'th class
        # take
        for i in random_num:
            test_image.append(images[i+min])
            test_label.append(labels[i+min])
        # pop
        for counter, j in enumerate(random_num):
            images.pop(j+min-counter)
            labels.pop(j+min-counter)

        popped += quarter_label_size # keep track of popped images
        
    return images, labels, test_image, test_label        

#### PROBLEMS in split_datasets:
- index out of bound while popping the list
case 1:
```
index_to_pop = (4, 9, 3, 2, 7)
image_list = (0.a, 1.b, 2.c, 3.d, 4.e, 5.f, 6.g, 7.h, 8.i, 9.j) -> (idx.value)
first iter curr list = (0.a, 1.b, 2.c, 3.d, 4.e, 5.f, 6.g, 7.h, 8.i, 9.j)
    images.pop[4]
    (target = e | popped = e)

second iter curr list = (0.a, 1.b, 2.c, 3.d, 4.f, 5.g, 6.h, 7.i, 8.j) 
    images.pop[9-0]
    (target = j | popped = ?) -> error (max index = 8)
```
tried:
- implementing counter -> if one data is popped, then the index will-1 (counter). since i dont change the true value (i just pop through the index), the counter must increase each time. (problem solved)

#### Implementing counter causes:
case 2:
```
index_to_pop = (4, 9, 3, 2, 7)
image_list = (0.a, 1.b, 2.c, 3.d, 4.e, 5.f, 6.g, 7.h, 8.i, 9.j) -> (idx.value)
first iter (counter=0) curr list = (0.a, 1.b, 2.c, 3.d, 4.e, 5.f, 6.g, 7.h, 8.i, 9.j)
    images.pop[4-0]
    (target = e | popped = e)

second iter (counter=1) curr list = (0.a, 1.b, 2.c, 3.d, 4.f, 5.g, 6.h, 7.i, 8.j)
    images.pop[9-1]
    (target = j | popped = j)

third iter (counter=2) curr list = (0.a, 1.b, 2.c, 3.d, 4.f, 5.g, 6.h, 7.i)
    images.pop[3-2]
    (target = d | popped = b) -> logic error
```
hencewhy I added "random_num.sort()" 

In [3]:
dataset_path = "Dataset"
image_datasets, label_datasets, num = get_dataset(dataset_path)
print(f"{len(image_datasets)} - {len(label_datasets)}")

200 - 200


In [4]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from matplotlib import pyplot as plt
from keras.utils import to_categorical

to-test:
+ resize => must
+ image augmentation => high

+ detect face then crop -> into square (use haar cascade)

## Preprocessing (if needed):
- /255 regularization
- find max widht & max height
- change label to hot encoded

(if needed)
- image augmentation 
- change channel size to 1

In [7]:
# find max height & width in picture
def find_max_image(image_dataset):
    max_height = 0
    max_width = 0
    for gambar in image_dataset:
        temp_height = gambar.shape[0]
        temp_width = gambar.shape[1]

        if temp_width > max_width:
            max_width = temp_width
        if temp_height > max_height:
            max_height = temp_height
    return max_height, max_width

# find min height & width
def find_min_image(image_dataset):
    min_height = 99999999
    min_width = 99999999
    for gambar in image_dataset:
        temp_height = gambar.shape[0]
        temp_width = gambar.shape[1]

        if temp_width < min_width:
            min_width = temp_width
        if temp_height < min_height:
            min_height = temp_height
    return min_height, min_width


In [9]:
max_height, max_width = find_max_image(image_datasets)
min_h, min_w = find_min_image(image_datasets)
print(f"{max_height}, {max_width}")

842, 474


#### Detect face (can use haar cascade)

In [ ]:
## Detect face
def detect_face():
    print("Hwllo")

def draw_bounding_accuracy():
    print("Hello")

#### Padding

In [9]:
## full padding -> resourcely expensive (kinda brute forcing)
def pad_image(image, desired_height, desired_width):
    height, width, dim = image.shape
    top = int((desired_height-height)/2)
    left = int((desired_width-width)/2)
    down = top
    right = left

    if (height % 2 == 1):
        down+=1
    if (width % 2 == 1):
        right+=1

    padded = np.pad(image, ((top, down), (left, right), (0, 0)), constant_values=0)

    return padded

def iterate_for_pad(images, desired_height, desired_width):
    padded_list = []
    for x in images:
        padded_list.append(pad_image(x, desired_height, desired_width))
    
    return padded_list

In [10]:
padded_images = iterate_for_pad(image_datasets, max_height, max_width)

---

#### Split dataset

In [11]:
train_image, train_label, test_image, test_label = split_datasets(padded_images, label_datasets, num)
train_image = np.array(train_image)
test_image = np.array(test_image)

In [12]:
NUM_CLASS = len(os.listdir(dataset_path))
INPUT_SHAPE = train_image[0].shape
print(INPUT_SHAPE)

(842, 474, 3)


In [13]:
train_label = to_categorical(train_label, NUM_CLASS)
test_label = to_categorical(test_label, NUM_CLASS)

In [14]:
model = Sequential([
    # input layer
    layers.Input(shape=INPUT_SHAPE),
    # convolutional & pooling
    layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    layers.MaxPool2D(2,2),
    # flatten
    layers.Flatten(),
    # output
    layers.Dense(64, activation="relu"),
    layers.Dense(NUM_CLASS, activation="softmax")
])

# adam (learning rate)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [15]:
## ini training
history = model.fit(
    train_image,
    train_label,
    epochs=10,
)

loss, accuracy = model.evaluate(test_image, test_label, verbose=0)

Epoch 1/10


In [ ]:
print("loss: {loss}, acc: {accuracy}")